In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.signal as signal

import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
## Signals are defined by a rate in rpm and an initial phase (in degrees).  
## Signals are pointed at the detector pi phase

def rpm2radpersec(rpm):
    return rpm/60*(2*np.pi)

signals = [[12, 0], [6,43] ]

## Transform signals to rad/sec and radians phase
signals = [[rpm2radpersec(a[0]), np.radians(a[1])] for a in signals]

In [3]:
sband_hz=10
buffer_secs=60

np.random.seed(0)

detector_mean = np.pi
detector_var  = 0.1
p_detection_max = 1.0
p_detection_random = 0.00

buffer_len = sband_hz*buffer_secs

buffer = np.zeros((buffer_len))
t = np.arange(0,buffer_len) / sband_hz

plt.figure(figsize=(16, 16))

for i,sig in enumerate(signals):
    angle = (sig[0]*t + signal[1])%(2*np.pi)

    p_detect = stats.norm.pdf(angle,detector_mean,detector_var) * p_detection_max / stats.norm.pdf(0,0,detector_var)
    prob = np.random.random(buffer_len)
    
    ## Add a constant probability of random noise
    p_detect = np.clip( p_detect, p_detection_random, None )

    plt.subplot(len(signals)+1,1,i+1)
    plt.plot(p_detect, color='r')
    
    buffer = buffer+np.clip( np.sign(p_detect-prob), 0, 1 )
    
## Normalize down to 0-1
buffer = np.clip( buffer, 0, 1)
    
plt.subplot(len(signals)+1,1,len(signals)+1)
plt.plot(buffer)
plt.show()  

TypeError: 'module' object is not subscriptable

<Figure size 1152x1152 with 0 Axes>

# Detection

In [ ]:
c = np.correlate( buffer, buffer, mode="full" )

## Only look at one half
c = c[c.size//2:]

plt.figure(figsize=(16, 8))
plt.plot(c)
plt.show()

In [ ]:
f,pxx = signal.periodogram( buffer, sband_hz, window='hamming' )

plt.figure(figsize=(16, 8))
plt.semilogy(f,pxx)
plt.xlabel("frequency Hz")
plt.show()
